In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

법정동코드 전체자료는 [행정표준관리시스템](https://www.code.go.kr/stdcode/regCodeL.do) 에서 압축파일을 다운받을 수 있다. <br>
Pandas의 read_csv() 함수는 zip 압축파일을 곧바로 읽어올 수 있으며, \t는 \[tab\]을 뜻한다.

In [2]:
df_cd = pd.read_csv("법정동코드 전체자료.zip", sep = "\t", encoding = "CP949")
df_cd.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


폐지여부 변수에 어떤 값이, 얼마나 있는지 확인

In [3]:
df_cd["폐지여부"].value_counts().to_frame().reset_index()

,index,폐지여부
0,폐지,25565
1,존재,20542


현재 사용하는(존재)만 필터링 하여 남긴다.

In [4]:
df_cd = df_cd.loc[df_cd["폐지여부"] == "존재", :]
df_cd.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


구 단위로 필터링하기 위해서 정규표현식(regular expression)을 사용한다. <br>
다음 코드의 정규표현식(^.*?0{5}$)은 처음부터(^) 몇 글자가 있던 관계없고(.*?), 0이 다섯번(0{5}) 반복되는 패턴이 마지막(\$)에 등장하는 자료를 잡아내기 위하여 사용되었다.

In [5]:
df_cd_sub = df_cd.loc[df_cd["법정동코드"].astype("str").str.match("^.*?0{5}$"), :]
df_cd_sub.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
94,1114000000,서울특별시 중구,존재
179,1117000000,서울특별시 용산구,존재
229,1120000000,서울특별시 성동구,존재


첫 다섯 글자를 뽑아내기 위해 str() 함수를 활용하여 숫자를 문자로 바꾸고, 다섯 번째 글자를 뽑아내는 코드(str(x)\[:5\])를 람다(lambda)함수로 작성하였다.

In [6]:
df_cd_sub.loc[:, "법정동코드"] = df_cd_sub.loc[:, "법정동코드"].apply(lambda x: str(x)[:5])
df_cd_sub.head(2)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,법정동코드,법정동명,폐지여부
0,11000,서울특별시,존재
1,11110,서울특별시 종로구,존재


예를 들어 인천이라는 글자가 들어간 법정동명을 뽑으려면 다음과 같이 작성한다.

In [7]:
df_cd_sub.loc[df_cd_sub["법정동명"].str.contains("인천"), :]

,법정동코드,법정동명,폐지여부
3280,28000,인천광역시,존재
3281,28110,인천광역시 중구,존재
3334,28140,인천광역시 동구,존재
3355,28177,인천광역시 미추홀구,존재
3363,28185,인천광역시 연수구,존재
3370,28200,인천광역시 남동구,존재
3415,28237,인천광역시 부평구,존재
3429,28245,인천광역시 계양구,존재
3454,28260,인천광역시 서구,존재
3477,28710,인천광역시 강화군,존재


수집을 위해 URL을 다음과 같이 구성한다.
* year_month: 조회 년월
* LAWD_CD: 법정동 코드 번호 다섯자리
* key: 발급받은 API key
* url: 원 주소. 중괄호 내부에는 변수에 할당된 값이 순서대로 입력된다.

In [8]:
year_month = "201907"
LAWD_CD = "46780"
key = "2jphw%2BFyE88rPf0gz5nICc9SyRbsIbYjD0VsGUiHplpkCUE%2FCQ02IP2vjacyoHETquky1enBlKXrTjpH%2BTsIZw%3D%3D"
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?LAWD_CD={0}&DEAL_YMD={1}&serviceKey={2}"
txt = requests.get(url.format(LAWD_CD, year_month, key))

requests의 get() 함수로 수집한 자료(txt 객체)를 BeautifulSoup의 함수를 사용하여 정제 및 필터링

In [9]:
xmlsoup = BeautifulSoup(txt.text, 'lxml-xml')
info_xml = xmlsoup.findAll("item")

첫 번째 내역 확인

In [10]:
info_xml[0]

<item><거래금액>     5,000</거래금액><건축년도>1991</건축년도><년>2019</년><도로명>태평길</도로명><도로명건물본번호코드>00004</도로명건물본번호코드><도로명건물부번호코드>00000</도로명건물부번호코드><도로명시군구코드>46780</도로명시군구코드><도로명일련번호코드>01</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>4670424</도로명코드><법정동>보성읍 보성리</법정동><법정동본번코드>0675</법정동본번코드><법정동부번코드>0000</법정동부번코드><법정동시군구코드>46780</법정동시군구코드><법정동읍면동코드>25021</법정동읍면동코드><법정동지번코드>1</법정동지번코드><아파트>보성</아파트><월>7</월><일>1</일><일련번호>46780-1</일련번호><전용면적>77.53</전용면적><지번>675</지번><지역코드>46780</지역코드><층>5</층></item>

각 거래를 정제하여 Pandas의 데이터프레임으로 만드는 과정

In [11]:
transaction = pd.DataFrame()
for t in info_xml:
    year  = t.find("년").text
    month = t.find("월").text
    day   = t.find("일").text
    built_year = t.find("건축년도").text
    dong = t.find("법정동").text
    apt_name = t.find("아파트").text
    size     = t.find("전용면적").text
    
    try:
        jibun = t.find("지번").text
    except:
        jibun = ""

    ji_code = t.find("지역코드").text
    floor   = t.find("층").text
    temp = pd.DataFrame(([[year, month, day, built_year, dong, apt_name, size, jibun, ji_code, floor]]), 
                        columns=["year", "month", "day", "build_year", "dong", "apt_name", "size", "jibun", "ji_code", "floor"])
    transaction = pd.concat([transaction, temp])

transaction = transaction.reset_index(drop = True)

In [12]:
transaction

,year,month,day,build_year,dong,apt_name,size,jibun,ji_code,floor
0,2019,7,1,1991,보성읍 보성리,보성,77.53,675,46780,5
1,2019,7,18,2008,보성읍 보성리,유화하이빌,84.74,688,46780,2
2,2019,7,22,2016,보성읍 보성리,보성센트럴뷰,84.8513,93-1,46780,2
3,2019,7,1,1999,보성읍 우산리,우산주공,49.85,631-1,46780,2
4,2019,7,3,1999,보성읍 우산리,우산주공,49.85,631-1,46780,8
5,2019,7,12,1999,보성읍 우산리,우산주공,49.85,631-1,46780,8
6,2019,7,3,1990,벌교읍 회정리,성진,79.85,58,46780,1
7,2019,7,4,1999,벌교읍 회정리,장미,59.37,486,46780,12
